In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('data/train_processed.csv')

In [9]:
df.head(10)

,Unnamed: 0,store_nbr,item_nbr,date,id,unit_sales,onpromotion,year,month,day,day_of_week,unit_sales_7d_avg
0,0,24,105574,2013-01-02,18790.0,12.0,False,2013,1,2,2,NaN
1,1,24,105574,2013-01-03,59692.0,1.0,False,2013,1,3,3,NaN
2,2,24,105574,2013-01-04,99664.0,3.0,False,2013,1,4,4,NaN
3,3,24,105574,2013-01-05,140805.0,4.0,False,2013,1,5,5,NaN
4,4,24,105574,2013-01-06,182800.0,7.0,False,2013,1,6,6,NaN
5,5,24,105574,2013-01-07,223895.0,3.0,False,2013,1,7,0,NaN
6,6,24,105574,2013-01-08,263107.0,3.0,False,2013,1,8,1,4.714286
7,7,24,105574,2013-01-09,302232.0,7.0,False,2013,1,9,2,4.000000
8,8,24,105574,2013-01-10,341031.0,3.0,False,2013,1,10,3,4.285714
9,9,24,105574,2013-01-11,379570.0,6.0,False,2013,1,11,4,4.714286


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35069356 entries, 0 to 35069355
Data columns (total 12 columns):
 #   Column             Dtype  
---  ------             -----  
 0   Unnamed: 0         int64  
 1   store_nbr          int64  
 2   item_nbr           int64  
 3   date               object 
 4   id                 float64
 5   unit_sales         float64
 6   onpromotion        object 
 7   year               int64  
 8   month              int64  
 9   day                int64  
 10  day_of_week        int64  
 11  unit_sales_7d_avg  float64
dtypes: float64(3), int64(7), object(2)
memory usage: 3.1+ GB


In [7]:
df['date'] = pd.to_datetime(df['date'])

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35069356 entries, 0 to 35069355
Data columns (total 12 columns):
 #   Column             Dtype         
---  ------             -----         
 0   Unnamed: 0         int64         
 1   store_nbr          int64         
 2   item_nbr           int64         
 3   date               datetime64[ns]
 4   id                 float64       
 5   unit_sales         float64       
 6   onpromotion        object        
 7   year               int64         
 8   month              int64         
 9   day                int64         
 10  day_of_week        int64         
 11  unit_sales_7d_avg  float64       
dtypes: datetime64[ns](1), float64(3), int64(7), object(1)
memory usage: 3.1+ GB


In [12]:
# select date after 1st march 2013
df = df[df['date'] > '2013-03-01']
# reindex the dataframe
df = df.reset_index(drop=True)

In [13]:
df.head

<bound method NDFrame.head of           Unnamed: 0  store_nbr  item_nbr       date           id  unit_sales  \
0                 59         24    105574 2013-03-02    2435397.0        11.0   
1                 60         24    105574 2013-03-03    2480417.0         3.0   
2                 61         24    105574 2013-03-04          NaN         0.0   
3                 62         24    105574 2013-03-05    2566767.0         2.0   
4                 63         24    105574 2013-03-06    2608657.0         1.0   
...              ...        ...       ...        ...          ...         ...   
33842859    35069351         51   2123791 2017-08-11          NaN         0.0   
33842860    35069352         51   2123791 2017-08-12          NaN         0.0   
33842861    35069353         51   2123791 2017-08-13          NaN         0.0   
33842862    35069354         51   2123791 2017-08-14          NaN         0.0   
33842863    35069355         51   2123791 2017-08-15  125491523.0         1.0  